## 1. Modelling the Environment

● Design a suitable representation for the robot's environment. Consider using a grid-based map where each cell represents a small area of the floor.
● Define states as specific grid cells that the robot can occupy.
● Identify possible actions the robot can take, such as moving forward, backward, turning left, or turning right (considering the robot's movement limitations).

In [1]:
class Environment:

# Function to create the grid environment.
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[0 for _ in range(width)] for _ in range(height)]
        self.robot_x = None
        self.robot_y = None

# Function to set the  robot's position on the grid.
    def set_robot_position(self, x, y):
        if self.robot_x is not None and self.robot_y is not None:
            self.grid[self.robot_y][self.robot_x] = 0
        self.robot_x = x
        self.robot_y = y
        self.grid[self.robot_y][self.robot_x] = "R"

# Functions to add Obstacles, Cleaning Target, Charging Station and Robot to the environment. 
    def add_obstacle(self, x, y):
        self.grid[y][x] = "O"

    def add_cleaning_target(self, x, y):
        self.grid[y][x] = "T"

    def add_charging_station(self, x, y):
        self.grid[y][x] = "C"

    def add_carpet(self, x, y):
        self.grid[y][x] = "W"

# Movement for Cleaning Robot
    def move_forward(self):
        if self.robot_y > 0 and self.grid[self.robot_y - 1][self.robot_x] != "O":
            self.set_robot_position(self.robot_x, self.robot_y - 1)

    def move_backward(self):
        if self.robot_y < self.height - 1 and self.grid[self.robot_y + 1][self.robot_x] != "O":
            self.set_robot_position(self.robot_x, self.robot_y + 1)

    def turn_left(self):
        if self.robot_x > 0 and self.grid[self.robot_y][self.robot_x - 1] != "O":
            self.set_robot_position(self.robot_x - 1, self.robot_y)

    def turn_right(self):
        if self.robot_x < self.width - 1 and self.grid[self.robot_y][self.robot_x + 1] != "O":
            self.set_robot_position(self.robot_x + 1, self.robot_y)

- *Environment* class represents the home environment of the cleaning robot.
- The *grid* attribute is a 2D list representing the floor plan of the room, where each element in the grid corresponds to one square meter of the floor. All cells are maked as empty(0).
- The methods *obstacle*, *cleaning_target* and *charging_station* are used to represent the obstacles to avoid, taget to clean and the charging station repectively.
- The methods *carpet* is used to represent the  carpet area, which requires more energy from the cleaning robot to traverse.
- You can create an instance of the Environment class with the desired width and height and then add obstacles, cleaning targets and charging stations as needed.

## 2. Cost Function and Heuristic

● Define a cost function that assigns a cost value to each action the robot takes. This could be a simple unit cost for each movement or consider factors like the type of terrain (carpet vs. hardwood) or energy expenditure for turns.

● Design a heuristic function h(x) that estimates the remaining distance for the robot to reach the cleaning target (represented by another specific grid cell). A simple option might be the Manhattan distance between the robot's current cell and the target cell, ignoring obstacles. However, explore a more sophisticated heuristic that considers the actual obstacles in the environment, penalising paths that would require the robot to navigate around them extensively.

In [2]:
# Cost Function
def cost_function(self, action):
    if action == "move_forward" or action == "move_backward":
        if self == "W":
            return 2 # Higher cost for moving on carpet
        elif self == 0:
            return 1 # Cost for moving on hardwood floor
        elif self == 1:
            return float('inf') #  Infinite cost for moving into an obstacle or wall - no movement possible
        return 1
    elif action == "turn_left" or action == "turn_right":
        return 0.5 # Cost for turning left or turning right

# Manhattan Distance Heuristic Function
def heuristic(self, target_x, target_y):
    distance_to_target = abs(target_x - self.robot_x) + abs(target_y - self.robot_y) # Manhattan Distance from current position to target position
    obstacles = sum(row.count("O") for row in self.grid) # For cleaning robot to consider obstacles and walls
    distance_to_target -= obstacles # Subtract 1 for each obstacle encountered for the robot to avoid
    carpet_area = sum(row.count("w") for row in self.grid)
    distance_to_target -= carpet_area+2 # Add 2 to account for the fact that the robot is already on a carpet square
    return distance_to_target

##  3. A Implementation and Testing

● Implement the A* search algorithm to find the optimal path for the robot to navigate from its starting position to clean every cell in the environment and finally reach the charging station (another designated cell).
● Test your implementation on various environments with different obstacle layouts and target locations.
● Analyse the impact of the chosen heuristic function on the efficiency of the robot's path. Compare the path lengths and total costs with the simple Manhattan distance heuristic vs. the more sophisticated obstacle-aware heuristic.

In [3]:
from queue import PriorityQueue
import matplotlib.pyplot as plt
import numpy as np

class Environment:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.grid = [[0 for _ in range(width)] for _ in range(height)]
        self.robot_x = None
        self.robot_y = None

    def set_robot_position(self, x, y):
        if self.robot_x is not None and self.robot_y is not None:
            self.grid[self.robot_y][self.robot_x] = 0
        self.robot_x = x
        self.robot_y = y
        self.grid[self.robot_y][self.robot_x] = "R"

    def add_obstacle(self, x, y):
        self.grid[y][x] = "O"

    def add_cleaning_target(self, x, y):
        self.grid[y][x] = "T"

    def add_charging_station(self, x, y):
        self.grid[y][x] = "C"

    def move_forward(self):
        if self.robot_y > 0 and self.grid[self.robot_y - 1][self.robot_x] != "O":
            self.set_robot_position(self.robot_x, self.robot_y - 1)

    def move_backward(self):
        if self.robot_y < self.height - 1 and self.grid[self.robot_y + 1][self.robot_x] != "O":
            self.set_robot_position(self.robot_x, self.robot_y + 1)

    def turn_left(self):
        if self.robot_x > 0 and self.grid[self.robot_y][self.robot_x - 1] != "O":
            self.set_robot_position(self.robot_x - 1, self.robot_y)

    def turn_right(self):
        if self.robot_x < self.width - 1 and self.grid[self.robot_y][self.robot_x + 1] != "O":
            self.set_robot_position(self.robot_x + 1, self.robot_y)

    def cost(self, action):
        if action == "move_forward" or action == "move_backward":
            return 1
        elif action == "turn_left" or action == "turn_right":
            return 0.5

    def heuristic(self, target_x, target_y, obstacles):
        return abs(target_x - self.robot_x) + abs(target_y - self.robot_y) + len(obstacles)

    def a_star_search(self, target_x, target_y):
        frontier = PriorityQueue()
        start_node = (self.robot_x, self.robot_y, 0, [])
        frontier.put((0, start_node))
        explored = set()

        while not frontier.empty():
            _, node = frontier.get()
            x, y, cost_so_far, path = node

            if (x, y) == (target_x, target_y):
                return path

            if (x, y) in explored:
                continue

            explored.add((x, y))

            actions = ["move_forward", "move_backward", "turn_left", "turn_right"]

            for action in actions:
                new_cost = cost_so_far + self.cost(action)
                if action == "move_forward":
                    new_x, new_y = x, y - 1
                elif action == "move_backward":
                    new_x, new_y = x, y + 1
                elif action == "turn_left":
                    new_x, new_y = x - 1, y
                elif action == "turn_right":
                    new_x, new_y = x + 1, y

                if 0 <= new_x < self.width and 0 <= new_y < self.height and self.grid[new_y][new_x] != "O":
                    new_path = path + [action]
                    priority = new_cost + self.heuristic(target_x, target_y, new_path)
                    frontier.put((priority, (new_x, new_y, new_cost, new_path)))

        return None
    
# Simple Grid
env1 = Environment(width=5, height=5)
env1.set_robot_position(0, 0)
env1.add_obstacle(1, 1)
env1.add_cleaning_target(4, 4)

# Complex Grid
env2 = Environment(width=9, height=9)
env2.set_robot_position(0, 0)
env2.add_obstacle(1, 0)
env2.add_obstacle(1, 1)
env2.add_obstacle(1, 2)
env2.add_obstacle(1, 3)
env2.add_obstacle(1, 4)
env2.add_obstacle(1, 5)
env2.add_obstacle(1, 6)
env2.add_obstacle(1, 7)
env2.add_obstacle(1, 8)
env2.add_obstacle(3, 6)
env2.add_obstacle(3, 7)
env2.add_obstacle(3, 8)
env2.add_obstacle(4, 6)
env2.add_obstacle(5, 6)
env2.add_obstacle(6, 6)
env2.add_obstacle(7, 6)
env2.add_obstacle(8, 6)
env2.add_obstacle(8, 7)
env2.add_obstacle(8, 8)
env2.add_cleaning_target(8, 8)

# Narrow Passage Grid
env3 = Environment(width=9, height=9)
env3.set_robot_position(0, 0)
env3.add_obstacle(7, 2)
env3.add_obstacle(7, 3)
env3.add_obstacle(7, 4)
env3.add_obstacle(7, 5)
env3.add_cleaning_target(8, 8)

# Maze Grid
env4 = Environment(width=9, height=9)
env4.set_robot_position(0, 0)
env4.add_obstacle(1, 1)
env4.add_obstacle(1, 2)
env4.add_obstacle(1, 3)
env4.add_obstacle(1, 4)
env4.add_obstacle(1, 5)
env4.add_obstacle(1, 6)
env4.add_obstacle(1, 7)
env4.add_obstacle(3, 3)
env4.add_obstacle(3, 6)
env4.add_obstacle(4, 3)
env4.add_obstacle(4, 6)
env4.add_obstacle(5, 3)
env4.add_obstacle(5, 6)
env4.add_obstacle(6, 3)
env4.add_obstacle(6, 4)
env4.add_obstacle(6, 5)
env4.add_obstacle(6, 6)
env4.add_cleaning_target(8, 8)

# Randomized Grid
np.random.seed(42)
env5 = Environment(width=9, height=9)
env5.set_robot_position(0, 0)
for _ in range(10):
    x = np.random.randint(0, 9)
    y = np.random.randint(0, 9)
    env5.add_obstacle(x, y)
env5.add_cleaning_target(8, 8)


def visualize_environment(env):
    grid = np.array(env.grid)
    plt.figure(figsize=(5, 5))
    plt.imshow(np.where(grid == "O", 0.5, np.where(grid == "T", 0.25, np.where(grid == "C", 0.75, 1))), cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.show()

visualize_environment(env1)
visualize_environment(env2)
visualize_environment(env3)
visualize_environment(env4)
visualize_environment(env5)

ModuleNotFoundError: No module named 'matplotlib'